In [48]:
import pandas as pd
import numpy as np
fn = '../src/data/raw/Community_2005-2017_Greenhouse_Gas_Emissions.csv'
emissions = pd.read_csv(fn)
emissions_trunc = pd.DataFrame(data=emissions.iloc[0:12])
emissions_trunc = pd.concat([emissions_trunc, emissions.iloc[13:14]], ignore_index=True)
emissions_trunc.index = emissions_trunc['Unnamed Column'].to_list()
emissions_trunc.index.names = ['Source']
bad_char = [',',r'[\(\)]']
emissions_trunc = emissions_trunc.replace(bad_char,'', regex=True).fillna(value=0)
emissions_trunc = emissions_trunc.apply(pd.to_numeric, errors='coerce')
emissions_trunc.iloc[1]= (
    emissions_trunc.iloc[1] + 
    emissions_trunc.iloc[2] + 
    emissions_trunc.iloc[3] + 
    emissions_trunc.iloc[9])
#non-res electricity includes large and small; and gov-non-city-owned electricity; and CTA rail elec
emissions_trunc.iloc[5]= (emissions_trunc.iloc[5] + 
                          emissions_trunc.iloc[6] + 
                          emissions_trunc.iloc[7])
#non-res gas includes commercial, industrial, fugitive
emissions_trunc.iloc[11] = emissions_trunc.iloc[11]*-1
small_row_names = ['Large Commercial Electricity', 
                   'Commercial Gas Consumption', 
                   'Fugitive Gas Emissions', 
                   'CTA Rail Electricty']
emissions_trunc = emissions_trunc.drop(small_row_names)
rename_dict = {'Small Commercial Electricity':'Non-Residential Electricity',
              'Industrial Gas Consumption': 'Non-Residential Gas',
              'Vehicle Miles Traveled (VMT)':'Vehicle Miles Traveled',
              'Renewable Energy Credits (RECs)':'Renewable Energy Credits'}
emissions_trunc = emissions_trunc.rename(rename_dict).iloc[:,1:]
emissions_trunc.to_csv('../src/data/interim/2005-2017_lessTruncated.csv')

trunc_emissions = pd.read_csv('../src/data/interim/2005-2017_lessTruncated.csv')
year_str = [str(int(blah)) for blah in np.linspace(2005, 2017, 13)]
idx_list = trunc_emissions.index
non_totals = trunc_emissions['Source'].copy()
type_list = [list(non_totals.values),
             [non_totals.str.split(' ')[i][-1].strip() for i in idx_list],
             [non_totals.str.split(' ')[i][0].strip() for i in idx_list]]
idx = pd.MultiIndex.from_arrays(type_list, names=['original', 'source', 'type'])
len_columns = [len(trunc_emissions.columns)]
emissions_lvld = trunc_emissions.copy()
emissions_lvld.index = idx
emissions_lvld.loc[:, 'id'] = np.array(np.arange(len(emissions_lvld)))
emissions_lvld.loc[:, 'source'] = type_list[0]
emissions_lvld.loc[:, 'type'] = [
    'Electricity', 
    'Electricity', 
    'Electricity',
    'Gas',
    'Gas',
    'Vehicle Miles Traveled',
    'Waste',
    'Renewable Energy Credit',
    'Municipal Mixed'
]

emissions_long = emissions_lvld.melt(col_level=0, id_vars=['id', 'source', 'type'], value_vars=year_str, var_name='Year')
emissions_long['id'] = emissions_long.index
emissions_long.rename(columns={'value':'MTCO2e', 'type':'Type'}, inplace=True)
emissions_long

,id,source,Type,Year,MTCO2e
0,0,Residential Electricity,Electricity,2005,154524.0
1,1,Non-Residential Electricity,Electricity,2005,385451.0
2,2,Government non-City-owned Electricty,Electricity,2005,641.0
3,3,Residential Gas Consumption,Gas,2005,114674.0
4,4,Non-Residential Gas,Gas,2005,225283.0
5,5,Vehicle Miles Traveled,Vehicle Miles Traveled,2005,121736.0
6,6,"Community Waste (garbage, recycling and organics)",Waste,2005,17000.0
7,7,Renewable Energy Credits,Renewable Energy Credit,2005,NaN
8,8,Estimated Municipal Operations GHG Emissions,Municipal Mixed,2005,24559.0
9,9,Residential Electricity,Electricity,2006,149040.0


In [49]:
emissions_long.to_csv('../src/data/interim/emissions2017_long_simple.csv')

In [36]:
emissions_trunc = emissions_trunc.fillna(0)
diff = emissions_trunc.sub(emissions_trunc.iloc[:,0], axis='index')
diff = diff.div(sum(emissions_trunc.iloc[:,0]))*100
diff.to_csv('../src/data/interim/2005-2017_lessTruncated_diff.csv')
trunc_emissions = pd.read_csv('../src/data/interim/2005-2017_lessTruncated_diff.csv')
trunc_emissions = trunc_emissions.drop('2005', axis=1)
trunc_emissions

,Source,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Residential Electricity,-0.525354,-0.395069,-0.709285,-1.790744,-0.451398,-0.629869,-0.099055,-1.362720,-1.425659,-3.172432,-3.850008,-4.160296
1,Non-Residential Electricity,1.317791,1.904072,0.827116,-0.477647,1.188081,0.494028,0.753448,-0.361253,-1.249871,-4.364920,-8.121717,-8.077841
2,Government non-City-owned Electricty,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008526
3,Residential Gas Consumption,0.202803,0.428215,1.209061,0.391620,0.004119,0.516924,0.516924,0.745688,-0.033721,0.360103,-0.824146,-0.496710
4,Non-Residential Gas,-0.221580,1.027812,2.052175,-0.536370,-2.473684,-1.244314,-1.244314,-0.260282,-3.677381,-0.248020,-1.125238,-1.089314
5,Vehicle Miles Traveled,0.000000,0.000000,0.000000,0.000000,-0.349853,-0.349853,-0.349853,-0.349853,-0.349853,-0.304636,0.039469,2.029854
6,"Community Waste (garbage, recycling and organics)",0.000000,0.000000,0.000000,0.000000,0.067920,0.067920,0.067920,0.067920,0.067920,-0.074147,0.116490,0.141301
7,Renewable Energy Credits,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-7.357348,-12.586170,-13.875413,-12.287665,-11.439952,-10.720129
8,Estimated Municipal Operations GHG Emissions,0.000000,0.000000,0.000000,0.000000,-0.508493,-0.508493,-0.508493,-0.508493,-0.508493,-1.328712,-1.572517,-1.572421


In [40]:
year_str = [str(int(blah)) for blah in np.linspace(2006, 2017, 12)]
idx_list = trunc_emissions.index
non_totals = trunc_emissions['Source'].copy()
type_list = [list(non_totals.values),
             [non_totals.str.split(' ')[i][-1].strip() for i in idx_list],
             [non_totals.str.split(' ')[i][0].strip() for i in idx_list]]
idx = pd.MultiIndex.from_arrays(type_list, names=['original', 'source', 'type'])
len_columns = [len(trunc_emissions.columns)]
emissions_lvld = trunc_emissions.copy()
emissions_lvld.index = idx
emissions_lvld.loc[:, 'id'] = np.array(np.arange(len(emissions_lvld)))
emissions_lvld.loc[:, 'source'] = type_list[0]
emissions_lvld.loc[:, 'type'] = [
    'Electricity', 
    'Electricity', 
    'Electricity',
    'Gas',
    'Gas',
    'Vehicle Miles Traveled',
    'Waste',
    'Renewable Energy Credit',
    'Municipal Mixed'
]
emissions_lvld = emissions_lvld.sort_index()
emissions_long = emissions_lvld.melt(col_level=0, id_vars=['id', 'source', 'type'], value_vars=year_str, var_name='Year')
emissions_long.sort_index()
emissions_long.rename(columns={'value':'dMTCO2e', 'type':'Type', 'source':'Source'}, inplace=True)
source = emissions_long['Source']
boo = ((source == 'Non-Residential Electricity') |
       (source == 'Non-Residential Gas') |
       (source == 'Renewable Energy Credits') |
       (source == 'Residential Electricity') |
       (source == 'Vehicle Miles Traveled'))
emissions_long = emissions_long[boo]
emissions_long = emissions_long.reset_index(drop = True)
emissions_long['id'] = emissions_long.index
emissions_long

,id,Source,Type,Year,dMTCO2e
0,0,Non-Residential Electricity,Electricity,2006,1.317791
1,1,Non-Residential Gas,Gas,2006,-0.221580
2,2,Renewable Energy Credits,Renewable Energy Credit,2006,0.000000
3,3,Residential Electricity,Electricity,2006,-0.525354
4,4,Vehicle Miles Traveled,Vehicle Miles Traveled,2006,0.000000
5,5,Non-Residential Electricity,Electricity,2007,1.904072
6,6,Non-Residential Gas,Gas,2007,1.027812
7,7,Renewable Energy Credits,Renewable Energy Credit,2007,0.000000
8,8,Residential Electricity,Electricity,2007,-0.395069
9,9,Vehicle Miles Traveled,Vehicle Miles Traveled,2007,0.000000


In [41]:
emissions_long.to_csv('../src/data/interim/emissions2017_long_diff_simple.csv')